In [4]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output
# imported base64 to display customer logo
import base64

import numpy as np
import pandas as pd


from pymongo import MongoClient



###########################
# Data Manipulation / Model
###########################
# changed to match my CRUD Python module file name and class name
from animalshelterCRUD import AnimalShelter
# FIXED
# updated with my username and password and CRUD Python module name
username = "aacuser3"
password = "purple"
shelter = AnimalShelter(username, password)

# updated the next line to my method name and CRUD Python module class
df = pd.DataFrame.from_records(shelter.read({}))


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Dash DataTable Only')


# including image (logo) required for customer to see when dashboard is ran
image_filename = 'GraziosoSalvareLogo.png' # customer image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    # Included unique identifier
    html.Center(html.B(html.H1('Rosario Robinson CS-340 Dashboard'))),
    html.Hr(),
    # added customer logo and resized it
    html.Img(id='customer-image',src='data:image/png;base64,{}'.format(encoded_image.decode()),
             style={'width': '50%', 
                    'height': '50%'},
             alt='customer image'),
    # Adding buttons to filter AAC dataset 
    html.Div(className='row',
             style={'display' : 'flex'},
                  children=[
                      # Created buttons to help filter through animal data information based on rescue type
                          html.Button(id='submit-button-one', n_clicks=0, children='Water Rescue'),
                          html.Button(id='submit-button-two', n_clicks=0, children='Mountain Rescue'),
                          html.Button(id='submit-button-three', n_clicks=0, children='Disaster Rescue'),
                          html.Button(id='submit-button-four', n_clicks=0, children='Reset')
                  ]),
    dash_table.DataTable(
        id='datatable-interactivity', # id to use throughout functionalities
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        # Features included for interactive data table to make it user-friendly
        data=df.to_dict('records'),
        editable=False,
        #filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,
    ),
    html.Br(),
    html.Hr(),
    #This sets up the dashboard so that the chart and your geolocation chart are side-by-side
    html.Div(className='row',
            style={'display':'flex'},
                children=[
            html.Div(
                id='graph-id',
                className='col s12 m6',
            ),
            html.Div(
                id='map-id',
                className='col s12 m6'
            )
        ])

])

#############################################
# Interaction Between Components / Controller
#############################################

# Data Table Buttons
# Changed callback output and inputs to help display data and properly filter through table
@app.callback(Output('datatable-interactivity','data'),
              [Input('submit-button-one', 'n_clicks'),Input('submit-button-two', 'n_clicks'),
               Input('submit-button-three', 'n_clicks'),Input('submit-button-four', 'n_clicks')
              ])
# button function to click through and display information required
def on_click(bt1,bt2,bt3,bt4):
    # start case
    # button one displays for Water Rescues
    if (bt1):
        df = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog","breed":
             {"$in":["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]},
             "sex_upon_outcome":"Intact Female",
             "age_upon_outcome_in_weeks":{"$gte":26},
             "age_upon_outcome_in_weeks":{"$lte":156}}))
    # button two displays for Mountain Rescues
    elif (bt2):
        df = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog","breed":
             {"$in":["German Shepherd","Alaskan Malamute","Old English Sheepdog","Siberian Huskey","Rottweiler"]},
             "sex_upon_outcome":"Intact Male",
             "age_upon_outcome_in_weeks":{"$gte":26},
             "age_upon_outcome_in_weeks":{"$lte":156}}))
    # button three displays for Disaster Rescues
    elif (bt3):
        df = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog","breed":
             {"$in":["Doberman Pinscher","German Shepherd","Golden Retriever","Bloodhound","Rottweiler"]},
             "sex_upon_outcome":"Intact Female",
             "age_upon_outcome_in_weeks":{"$gte":20},
             "age_upon_outcome_in_weeks":{"$lte":300}}))
    # button four resets back to all data
    if (bt4):
        df = pd.DataFrame.from_records(shelter.read({}))
        
        
    return df.to_dict('records')



@app.callback(Output('datatable-interactivity', 'style_data_conditional'),
              [Input('datatable-interactivity', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Pie chart section
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-interactivity', "derived_viewport_data")])

def update_graphs(viewData):
    # FIXED
    dff = pd.DataFrame.from_dict(viewData)
    # created two variables to take in the inputs from the animal shelter data
    # help from: https://medium.com/analytics-vidhya/a-guide-to-interactive-data-visualizations-with-python-plotly-c1b949eeb1f
    new_names = dff['breed'].value_counts().keys().tolist()
    new_values = dff['breed'].value_counts().tolist()
    # Created pie chart with additional features such as adding a hole, various colors, and resizing
    return [
        dcc.Graph(            
            figure = px.pie(
                data_frame=dff, 
                names = new_names, 
                values = new_values,
                hole=.3,
                # https://plotly.com/python/discrete-color/
                color_discrete_sequence=px.colors.qualitative.Dark2,
                width=800, 
                height=500   
            )
        )
    ]


@app.callback(
    Output('map-id', "children"),
    [Input('datatable-interactivity', "derived_viewport_data")])

def update_map(viewData):
# FIXED 
# Added in the code for my geolocation chart
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Updated this section to use loc() method to update location when button is pressed
            # Help from: https://www.geeksforgeeks.org/difference-between-loc-and-iloc-in-pandas-dataframe/
            dl.Marker(position=[dff.loc[0,'location_lat'],dff.loc[0,'location_long']], children=[
                dl.Tooltip(dff['breed']),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.loc[0,'name'])
                ])
            ])
        ])
    ]








app
